In [1]:
import os
import random
import cv2
import numpy as np
from tqdm import tqdm
from skimage.transform import resize
from PIL import Image
import shutil
import random
import albumentations as A
import nibabel as nib
import matplotlib.pyplot as plt
from natsort import natsorted

f:\Yashesh\Anaconda\envs\deeplabv3_env\lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.3 (you have 1.4.23). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
def load_nifti_data(nifti_path):
    img = nib.load(nifti_path)
    try:
        data = img.get_fdata()
    except AttributeError:
        data = img.get_data()
    return data


def z_score_normalization(data):
    mean = np.mean(data)
    std = np.std(data)
    if std == 0:
        return data
    normalized_data = (data - mean) / std
    return normalized_data


def process_masks(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for filename in os.listdir(input_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):  # Add other formats if needed
            img_path = os.path.join(input_folder, filename)
            img = Image.open(img_path)

            # Convert to grayscale if not already
            if img.mode != 'L':
                img = img.convert('L')

            # Get unique colors in the image
            colors = set(img.getdata())

            # Check the number of unique colors
            if len(colors) == 5:  # Only two colors (black and white)
                # Change white to grey
                new_img = img.point(lambda p: 128 if p == 255 else p)
                new_img.save(os.path.join(output_folder, filename))
            else:
                # Save the original image unchanged
                img.save(os.path.join(output_folder, filename))


def load_and_save_slices(nifti_dir, output_dir, mask_final_output_dir, limit=1):
    case_folders = [case for case in os.listdir(nifti_dir) if os.path.isdir(os.path.join(nifti_dir, case))]

    for i, case in enumerate(case_folders):
        if i >= limit:
            break

        nii_file = os.path.join(nifti_dir, case, 'imaging.nii.gz')
        mask_file = os.path.join(nifti_dir, case, 'segmentation.nii.gz')

        if os.path.exists(nii_file) and os.path.exists(mask_file):
            print(f"Processing {nii_file} and {mask_file}...")

            data = load_nifti_data(nii_file)
            mask_data = load_nifti_data(mask_file)

            normalized_data = z_score_normalization(data)

            num_slices = normalized_data.shape[0]

            for slice_index in range(num_slices):
                mask_slice = mask_data[slice_index, :, :]

                if np.any(mask_slice > 0):
                    slice_data = normalized_data[slice_index, :, :]
                    resized_slice = resize(slice_data, (256, 256), anti_aliasing=True)
                    resized_mask = resize(mask_slice, (256, 256), anti_aliasing=False, preserve_range=True)

                    slice_filename = f'slice_{case}_{slice_index}.png'
                    mask_filename = f'slice_{case}_{slice_index}_mask.png'

                    slice_path = os.path.join(output_dir, slice_filename)
                    mask_path = os.path.join(mask_final_output_dir, mask_filename)

                    os.makedirs(output_dir, exist_ok=True)
                    os.makedirs(mask_final_output_dir, exist_ok=True)

                    plt.imsave(slice_path, resized_slice, cmap='gray')
                    plt.imsave(mask_path, resized_mask, cmap='gray')
                    print(f"Image and Mask Slice {slice_index} saved to {slice_path} and {mask_path}")

            # Process the masks after saving them
            process_masks(mask_final_output_dir, mask_final_output_dir)

        else:
            print(f"Files not found for {case}")

nifti_dir = r'E:\DeepLabv3+\DeepLabV3Plus-Pytorch\KiTS23\kits23\dataset'
slice_output_dir = r'E:\DeepLabv3+\DeepLabV3Plus-Pytorch\KiTS23\data\JPEGImages'
mask_output_dir = r'E:\DeepLabv3+\DeepLabV3Plus-Pytorch\KiTS23\data\Annotations'

load_and_save_slices(nifti_dir, slice_output_dir, mask_output_dir, limit=5)


Files not found for .ipynb_checkpoints
Processing E:\DeepLabv3+\DeepLabV3Plus-Pytorch\KiTS23\kits23\dataset\case_00000\imaging.nii.gz and E:\DeepLabv3+\DeepLabV3Plus-Pytorch\KiTS23\kits23\dataset\case_00000\segmentation.nii.gz...
Image and Mask Slice 90 saved to E:\DeepLabv3+\DeepLabV3Plus-Pytorch\KiTS23\data\JPEGImages\slice_case_00000_90.png and E:\DeepLabv3+\DeepLabV3Plus-Pytorch\KiTS23\data\Annotations\slice_case_00000_90_mask.png
Image and Mask Slice 91 saved to E:\DeepLabv3+\DeepLabV3Plus-Pytorch\KiTS23\data\JPEGImages\slice_case_00000_91.png and E:\DeepLabv3+\DeepLabV3Plus-Pytorch\KiTS23\data\Annotations\slice_case_00000_91_mask.png
Image and Mask Slice 92 saved to E:\DeepLabv3+\DeepLabV3Plus-Pytorch\KiTS23\data\JPEGImages\slice_case_00000_92.png and E:\DeepLabv3+\DeepLabV3Plus-Pytorch\KiTS23\data\Annotations\slice_case_00000_92_mask.png
Image and Mask Slice 93 saved to E:\DeepLabv3+\DeepLabV3Plus-Pytorch\KiTS23\data\JPEGImages\slice_case_00000_93.png and E:\DeepLabv3+\DeepLabV3

In [4]:
def split_data(input_slices_dir, input_masks_dir, output_dir, train_size=0.7, val_size=0.15):
    # Create output directories
    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'val')
    test_dir = os.path.join(output_dir, 'test')

    for dir in [train_dir, val_dir, test_dir]:
        os.makedirs(dir, exist_ok=True)
        os.makedirs(os.path.join(dir, 'slices'), exist_ok=True)
        os.makedirs(os.path.join(dir, 'masks'), exist_ok=True)

    # Get list of files and create pairs of slices and masks
    slice_files = [f for f in os.listdir(input_slices_dir) if f.endswith('.png')]
    mask_files = [f for f in os.listdir(input_masks_dir) if f.endswith('.png')]

    # Create pairs of slices and their corresponding masks
    pairs = [(slice_file, mask_file) for slice_file, mask_file in zip(slice_files, mask_files)]

    # Shuffle the pairs
    random.seed(42)  # For reproducibility
    random.shuffle(pairs)

    # Calculate the split indices
    total = len(pairs)
    train_end = int(train_size * total)
    val_end = train_end + int(val_size * total)

    # Split the pairs into train, validation, and test sets
    train_pairs = pairs[:train_end]
    val_pairs = pairs[train_end:val_end]
    test_pairs = pairs[val_end:]

    # Function to copy files to the appropriate directory
    def copy_files(pairs, target_dir):
        for slice_file, mask_file in pairs:
            # Copy slice and mask
            shutil.copy(os.path.join(input_slices_dir, slice_file), os.path.join(target_dir, 'slices', slice_file))
            shutil.copy(os.path.join(input_masks_dir, mask_file), os.path.join(target_dir, 'masks', mask_file))

    # Copy files to their respective directories
    copy_files(train_pairs, train_dir)
    copy_files(val_pairs, val_dir)
    copy_files(test_pairs, test_dir)

    print("Data split completed!")

# Example usage
input_slices_dir = slice_output_dir  # Directory with original slice images
input_masks_dir = mask_output_dir  # Directory with original mask images
output_dir_split = r'D:\Pawan\Project\ImgSeg\Deeplabv3+\DeepLabV3Plus-Pytorch\kits23\split_data'  # Directory for the split data

split_data(input_slices_dir, input_masks_dir, output_dir_split)


Data split completed!


In [14]:
def overlay_slices_with_masks(slice_folder, mask_folder, overlay_folder):
    os.makedirs(overlay_folder, exist_ok=True)
    
    # Loop through slices in the folder
    for slice_filename in os.listdir(slice_folder):
        if slice_filename.endswith('.png'):
            slice_path = os.path.join(slice_folder, slice_filename)
            mask_path = os.path.join(mask_folder, slice_filename.replace('.png', '_mask.png'))
            
            # Ensure both slice and mask exist
            if os.path.exists(slice_path) and os.path.exists(mask_path):
                # Load slice and mask
                slice_img = Image.open(slice_path).convert('RGBA')
                mask_img = Image.open(mask_path).convert('RGBA')
                
                # Resize mask to match the slice dimensions if needed
                if slice_img.size != mask_img.size:
                    mask_img = mask_img.resize(slice_img.size)
                
                # Overlay mask on top of the slice
                overlay_img = Image.blend(slice_img, mask_img, alpha=0.5)
                
                # Save the overlay image
                overlay_path = os.path.join(overlay_folder, slice_filename)
                overlay_img.save(overlay_path)
                print(f"Overlay saved to {overlay_path}")
            else:
                print(f"Missing slice or mask for {slice_filename}")



nifti_dir = r'D:\Pawan\Project\ImgSeg\Deeplabv3+\DeepLabV3Plus-Pytorch\kits23\dataset'
slice_output_dir = r'D:\Pawan\Project\ImgSeg\Deeplabv3+\DeepLabV3Plus-Pytorch\kits23\output_slices'
mask_output_dir = r'D:\Pawan\Project\ImgSeg\Deeplabv3+\DeepLabV3Plus-Pytorch\kits23\output_masks'
overlay_folder = r'D:\Pawan\Project\ImgSeg\Deeplabv3+\DeepLabV3Plus-Pytorch\kits23\Overlay'

overlay_slices_with_masks(slice_output_dir, mask_output_dir, overlay_folder)

Overlay saved to D:\Pawan\Project\ImgSeg\Deeplabv3+\DeepLabV3Plus-Pytorch\kits23\Overlay\slice_case_00000_100.png
Overlay saved to D:\Pawan\Project\ImgSeg\Deeplabv3+\DeepLabV3Plus-Pytorch\kits23\Overlay\slice_case_00000_101.png
Overlay saved to D:\Pawan\Project\ImgSeg\Deeplabv3+\DeepLabV3Plus-Pytorch\kits23\Overlay\slice_case_00000_102.png
Overlay saved to D:\Pawan\Project\ImgSeg\Deeplabv3+\DeepLabV3Plus-Pytorch\kits23\Overlay\slice_case_00000_103.png
Overlay saved to D:\Pawan\Project\ImgSeg\Deeplabv3+\DeepLabV3Plus-Pytorch\kits23\Overlay\slice_case_00000_104.png
Overlay saved to D:\Pawan\Project\ImgSeg\Deeplabv3+\DeepLabV3Plus-Pytorch\kits23\Overlay\slice_case_00000_105.png
Overlay saved to D:\Pawan\Project\ImgSeg\Deeplabv3+\DeepLabV3Plus-Pytorch\kits23\Overlay\slice_case_00000_106.png
Overlay saved to D:\Pawan\Project\ImgSeg\Deeplabv3+\DeepLabV3Plus-Pytorch\kits23\Overlay\slice_case_00000_107.png
Overlay saved to D:\Pawan\Project\ImgSeg\Deeplabv3+\DeepLabV3Plus-Pytorch\kits23\Overlay

In [24]:
def create_video_from_overlays(overlay_folder, video_output_folder, fps=10):
    
    os.makedirs(video_output_folder, exist_ok=True)
    
    # Group overlay images by cases
    case_dict = {}
    for filename in os.listdir(overlay_folder):
        if filename.endswith('.png'):
            # Extract case name based on the file naming convention
            case_name = filename.split('_')[1] + "_" + filename.split('_')[2]  # Assuming format: slice_case_XXXXX_###.png
            # print(case_name)
            if case_name not in case_dict:
                case_dict[case_name] = []
            case_dict[case_name].append(filename)
    
    # Create a video for each case
    for case_name, file_list in case_dict.items():
        # Sort files naturally (e.g., slice_case_00000_1.png, slice_case_00000_2.png, ...)
        sorted_files = natsorted(file_list)
        
        # Read the first image to get the video dimensions
        first_image_path = os.path.join(overlay_folder, sorted_files[0])
        frame = cv2.imread(first_image_path)
        height, width, _ = frame.shape

        # Define the video writer
        video_path = os.path.join(video_output_folder, f'{case_name}_overlay_video.mp4')
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
        video_writer = cv2.VideoWriter(video_path, fourcc, fps, (width, height))
        
        # Add each image to the video
        for image_name in sorted_files:
            image_path = os.path.join(overlay_folder, image_name)
            frame = cv2.imread(image_path)
            video_writer.write(frame)

        # Release the video writer
        video_writer.release()
        print(f"Video saved for case {case_name}: {video_path}")


# Paths
overlay_folder = r'D:\Pawan\Project\ImgSeg\Deeplabv3+\DeepLabV3Plus-Pytorch\kits23\Overlay'
video_output_folder = r'D:\Pawan\Project\ImgSeg\Deeplabv3+\DeepLabV3Plus-Pytorch\kits23\Videos'

# Create videos
create_video_from_overlays(overlay_folder, video_output_folder, fps=30)


case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000
case_00000